In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.cross_validation import KFold,cross_val_score,train_test_split
import tensorflow as tf
from dateutil import parser

plt.rcParams['figure.figsize'] = (10,8)

G:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
G:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
billboard = pd.read_csv("billboard_chart_data.csv")

In [3]:
billboard = billboard[['title','artist','rank', 'date']]

In [4]:
billboard.head()

,title,artist,rank,date
0,God's Plan,Drake,1,2018-02-17
1,Perfect,Ed Sheeran,2,2018-02-17
2,Finesse,Bruno Mars & Cardi B,3,2018-02-17
3,Havana,Camila Cabello Featuring Young Thug,4,2018-02-17
4,Rockstar,Post Malone Featuring 21 Savage,5,2018-02-17


In [5]:
track = pd.read_csv("Full_data.csv")

In [6]:
track.head()

,id,title,artist,track_id,artist_id,rank,hit,artist_popularity,followers,popularity,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1,God's Plan,Drake,2XW4DbS6NddZxRPm5rMCeY,3TVXtAsR1Inumwj472S9r4,1,Yes,100,18311322,99,...,0.454,7,-9.488,1,0.0963,0.0244,0.000056,0.4980,0.344,77.170
1,2,Perfect,Ed Sheeran,0tgVpDi06FyKpA1z0VMD4v,6eUKZXaKkcviH0Ku9w2n3V,1,Yes,96,21431294,95,...,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050
2,3,Finesse,Bruno Mars & Cardi B,3Vo4wInECJQuz9BIBMOu8i,0du5cEVh5yTK9QJze8zA0C,3,Yes,91,11594549,95,...,0.859,5,-4.877,0,0.0996,0.0185,0.000000,0.0215,0.926,105.115
3,4,Havana,Camila Cabello Featuring Young Thug,1rfofaqEpACxVEHIZBJe6W,4nDoRrQiYLoBzwC5BhVJzF,1,Yes,91,3144195,97,...,0.523,2,-4.333,1,0.0300,0.1840,0.000036,0.1320,0.394,104.988
4,5,Rockstar,Post Malone Featuring 21 Savage,0OAAAdiHJKa2wlCKqaYXV7,246dkjvS1zLTtiykXe5h60,1,Yes,96,3309267,90,...,0.535,5,-6.637,0,0.0776,0.1300,0.000130,0.1430,0.127,159.764


In [7]:
merge_data = pd.merge(billboard,track,on=['title','artist'])

In [8]:
del merge_data['rank_y']
del merge_data['hit']

In [9]:
merge_data.head()

,title,artist,rank_x,date,id,track_id,artist_id,artist_popularity,followers,popularity,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,God's Plan,Drake,1,2018-02-17,1,2XW4DbS6NddZxRPm5rMCeY,3TVXtAsR1Inumwj472S9r4,100,18311322,99,...,0.454,7,-9.488,1,0.0963,0.0244,0.000056,0.498,0.344,77.17
1,God's Plan,Drake,1,2018-02-10,1,2XW4DbS6NddZxRPm5rMCeY,3TVXtAsR1Inumwj472S9r4,100,18311322,99,...,0.454,7,-9.488,1,0.0963,0.0244,0.000056,0.498,0.344,77.17
2,God's Plan,Drake,1,2018-02-03,1,2XW4DbS6NddZxRPm5rMCeY,3TVXtAsR1Inumwj472S9r4,100,18311322,99,...,0.454,7,-9.488,1,0.0963,0.0244,0.000056,0.498,0.344,77.17
3,Perfect,Ed Sheeran,2,2018-02-17,2,0tgVpDi06FyKpA1z0VMD4v,6eUKZXaKkcviH0Ku9w2n3V,96,21431294,95,...,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.106,0.168,95.05
4,Perfect,Ed Sheeran,2,2018-02-10,2,0tgVpDi06FyKpA1z0VMD4v,6eUKZXaKkcviH0Ku9w2n3V,96,21431294,95,...,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.106,0.168,95.05


In [10]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89689 entries, 0 to 89688
Data columns (total 21 columns):
title                89689 non-null object
artist               89689 non-null object
rank_x               89689 non-null int64
date                 89689 non-null object
id                   89689 non-null int64
track_id             89689 non-null object
artist_id            89689 non-null object
artist_popularity    89689 non-null int64
followers            89689 non-null int64
popularity           89689 non-null int64
danceability         89689 non-null float64
energy               89689 non-null float64
key                  89689 non-null int64
loudness             89689 non-null float64
mode                 89689 non-null int64
speechiness          89689 non-null float64
acousticness         89689 non-null float64
instrumentalness     89689 non-null float64
liveness             89689 non-null float64
valence              89689 non-null float64
tempo                89689 non

In [11]:
merge_data = merge_data.sort_values(by=['date','rank_x'],ascending=True)

In [12]:
merge_data.to_csv("merge_data.csv", index = None)

In [13]:
df = pd.read_csv("merge_data.csv")
#df.reset_index()

In [14]:
print(df)

                                  title  \
0                        I'm Your Angel   
1          Nobody's Supposed To Be Here   
2                  Doo Wop (That Thing)   
3                   From This Moment On   
4                        Love Like This   
5                               Lullaby   
6                 ...Baby One More Time   
7                        Have You Ever?   
8                       The First Night   
9                                  Iris   
10                            This Kiss   
11                             Trippin'   
12                How Deep Is Your Love   
13                               Jumper   
14                         Save Tonight   
15                                Hands   
16                             One Week   
17                              Thank U   
18                  My Favorite Mistake   
19                                Slide   
20                              I'll Be   
21               Are You That Somebody?   
22         

In [15]:
peak = []
for i in range(1, len(df)+1):
    tmp = []
    c = df['rank_x'][i]
    if c < 30:
        tmp.append("Yes")
    else:
        tmp.append("No")
    peak.append(tmp)
df_peak = pd.DataFrame(peak)
df_peak.columns = ['hit']


In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89689 entries, 0 to 89688
Data columns (total 21 columns):
title                89689 non-null object
artist               89689 non-null object
rank_x               89689 non-null int64
date                 89689 non-null object
id                   89689 non-null int64
track_id             89689 non-null object
artist_id            89689 non-null object
artist_popularity    89689 non-null int64
followers            89689 non-null int64
popularity           89689 non-null int64
danceability         89689 non-null float64
energy               89689 non-null float64
key                  89689 non-null int64
loudness             89689 non-null float64
mode                 89689 non-null int64
speechiness          89689 non-null float64
acousticness         89689 non-null float64
instrumentalness     89689 non-null float64
liveness             89689 non-null float64
valence              89689 non-null float64
tempo                89689 non

In [17]:
df_peak.head()

,hit
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes


In [18]:
print(df_peak)

       hit
0      Yes
1      Yes
2      Yes
3      Yes
4      Yes
5      Yes
6      Yes
7      Yes
8      Yes
9      Yes
10     Yes
11     Yes
12     Yes
13     Yes
14     Yes
15     Yes
16     Yes
17     Yes
18     Yes
19     Yes
20     Yes
21     Yes
22     Yes
23     Yes
24      No
25      No
26      No
27      No
28      No
29      No
...    ...
89658   No
89659   No
89660   No
89661   No
89662   No
89663   No
89664   No
89665   No
89666   No
89667   No
89668   No
89669   No
89670   No
89671   No
89672   No
89673   No
89674   No
89675   No
89676   No
89677   No
89678   No
89679   No
89680   No
89681   No
89682   No
89683   No
89684   No
89685   No
89686   No
89687   No

[89688 rows x 1 columns]


In [19]:
result = pd.concat([df, df_peak], axis=1)

In [20]:
result.head()

,title,artist,rank_x,date,id,track_id,artist_id,artist_popularity,followers,popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,hit
0,I'm Your Angel,R. Kelly & Celine Dion,1,1998-12-12,7132,0QWFiyhOqFu9RP8qzP6z5L,2mxe0TnaNL039ysAj51xPQ,77,1465113,44,...,0,-7.260,1,0.0301,0.66900,0.000000,0.1100,0.266,112.681,Yes
1,Nobody's Supposed To Be Here,Deborah Cox,2,1998-12-12,7121,51QxenFmXlJXUN9mpvxlaL,601893mmW5hl1FBOykWZHG,55,160121,56,...,8,-5.300,1,0.0361,0.20200,0.000012,0.5950,0.399,119.957,Yes
2,Doo Wop (That Thing),Lauryn Hill,4,1998-12-12,7134,2Uu8IiLkLY0UXhCHka4Dlr,2Mu5NfyYm8n5iTomuKAEHl,70,816631,74,...,2,-8.926,0,0.2450,0.03930,0.000000,0.0923,0.495,99.935,Yes
3,From This Moment On,Shania Twain,5,1998-12-12,7161,7n2vETKqnlDt4670aVs5n5,5e4Dhzv426EvQe3aDb64jL,71,865667,50,...,7,-5.021,1,0.0271,0.38200,0.000000,0.1860,0.148,135.819,Yes
4,Love Like This,Faith Evans,7,1998-12-12,7140,7MQywXGHEev7JmwwIzMcao,5NDMothbpdpq2xHqSjrrWn,70,372926,60,...,0,-7.328,1,0.0616,0.00364,0.000000,0.0451,0.796,100.904,Yes


In [21]:
result.to_csv("billboard_charts.csv")

In [22]:
print(type(result['date'][1]))

<class 'str'>


In [23]:
dt = parser.parse(result['date'][1])
dt1 = parser.parse(result['date'][999])

In [24]:
print((dt1-dt).days)

77


In [25]:
time = []
for i in range(0, len(result)):
    tmp = []
    if i == 0:
        tmp.append("0")
    else:
        df1 = parser.parse(result['date'][0])
        df2 = parser.parse(result['date'][i])
        df = (df2-df1).days
        #print(df)
        tmp.append(df)
    time.append(tmp)
df_days = pd.DataFrame(time)
                                     

In [26]:
df_days.columns= ['days']

In [27]:
print(df_days)

       days
0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
...     ...
89659  7007
89660  7007
89661  7007
89662  7007
89663  7007
89664  7007
89665  7007
89666  7007
89667  7007
89668  7007
89669  7007
89670  7007
89671  7007
89672  7007
89673  7007
89674  7007
89675  7007
89676  7007
89677  7007
89678  7007
89679  7007
89680  7007
89681  7007
89682  7007
89683  7007
89684  7007
89685  7007
89686  7007
89687  7007
89688  7007

[89689 rows x 1 columns]


In [28]:
LSTM_data = pd.concat([result, df_days], axis=1)

In [29]:
LSTM_data.to_csv('LSTM_data.csv',index = None)